A characteristics of many neural networks such as densely connected networks or covnets is that they ave no memory. Each input shown to them is processed independently with no state kept in between inputs. In order to process sequences with these networks the whole sequence must be fed into the network at one and turn it to a single data point. They are what is called *feedforeward networks*.

*Recurrent Neural Networks* solve this problem by processesing sequences by iterating through the sequence elements and maintaining a *state* containing information relative to what it has seen so far. In essence an RNN is a type of neural network that has an internal loop and the state of the rnn is reset between processing two different independent sequences. This means that a single sequence is considered a single data point. What changes is that this data point is no longer processed in a single step, rather the network will internally loop over the sequence elements.